In [2]:
# importing dependencies
import pandas as pd
import numpy as np
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
import keras
import re
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet
from pywsd import disambiguate
from pywsd.similarity import max_similarity as maxsim
from nltk.wsd import lesk
import string

In [4]:
# disambiguate('I went to the bank to deposit my money')
disambiguate('love you', algorithm=maxsim, similarity_option='wup', keepLemmas=True)


[('love', 'love', Synset('love.n.04')), ('you', 'you', None)]

In [3]:
#stop word removal
from nltk.corpus import stopwords
stop={'a', 'of', 'on', 'the'} #add related stop word here
print(stop)

{'the', 'of', 'a', 'on'}


In [96]:
df_csv = pd.read_csv('datasets/only_cyberbyllying_formspring_datasets.csv')
df_csv1 = pd.read_csv('datasets/not_cyberbyllying_formspring_datasets.csv')
df_csv2 = pd.read_csv('datasets/ask_fm_data.csv')
df_csv3 = pd.read_csv('datasets/negation_detected.csv')

In [97]:
df_csv.head(10100)

,Unnamed: 0,q_a,label
0,0,Fuck Justin bieber. He's a fucking fag,1
1,1,its also funny how u stalked my whole twitter!...,1
2,2,Hello bitch! How's ur day going? Good I hope :...,1
3,3,Hey. Why you such a bitch?,1
4,4,how old are u ? And how do you know michael?,1
...,...,...,...
1021,1021,would your rather cheat or be cheated on?,0
1022,1022,you might think im evil but u r the evil,1
1023,1023,You're a bushwhacking alchy piece of shit scrub.,1
1024,1024,Your 44 years old? :O! Dumbass Pedifile.:D,1


In [98]:
#marge all data from files that has been read

q_a=[]
label=[]
for i in range(len(df_csv)):
    label.append(int(df_csv['label'][i]))
    q_a.append(df_csv['q_a'][i])
# print(len(q_a))
# print(len(label))

# for i in range(len(df_csv1)):
#     label.append(int(df_csv1['label'][i]))
#     q_a.append(df_csv1['q_a'][i])
# print(len(q_a))
# print(len(label))

for i in range(len(df_csv2)):
    label.append(int(df_csv2['label'][i]))
    q_a.append(df_csv2['q_a'][i])
print(len(q_a))
print(len(label))
    
# for i in range(len(df_csv3)):
#     label.append(int(df_csv3['label'][i]))
#     q_a.append(df_csv3['q_a'][i])

print(len(q_a))
print(len(label))


11024
11024
11024
11024


In [99]:
# text cleaning
w=list(string.ascii_lowercase)
QA=[]
for i in q_a:
    sen=i.replace('@USER','')
    sen=sen.replace('@user','')
    sen=sen.replace("#", ' ')
    sen=sen.replace("@", '')
    sen=sen.replace("pu**y", 'pussy')
    sen=sen.replace(" mofo ", ' motherfucker')
    sen=sen.replace("NEWLINE_TOKEN", " ")
    sen=sen.replace("TAB_TOKEN", " ")
    sen=sen.replace("??", '?')
    sen=sen.replace("? ?", '?')
    sen=sen.replace("??", '?')
    sen=sen.replace("? ?", '?')
    sen=sen.replace("? ?", '?')
    sen=sen.replace("? ?", '?')
    sen=sen.replace("?", ' ? ')
    sen=sen.replace("!", ' ! ')
    sen=sen.replace(" y ", ' why ')
    sen=sen.replace(" u ", ' you ')
    sen=sen.replace(" w ", ' with ')
    sen=sen.replace(" mf ", ' motherfucker ')
    sen=sen.replace(" IDGAF ", ' i dont give a fuck ')
    sen=sen.replace(" IDRC ", ' I Dont Really Care ')
    sen=sen.replace(" tf ", ' the fuck ')
    sen=sen.replace("wtf ", 'what the fuck ')
    sen=sen.replace(" yanno ", ' you know ')
    sen=sen.replace(" igt ", ' i know right ')
    sen=sen.replace(" r ", ' are ')
    sen=sen.replace(" tbh ", ' to be honest ')
    sen=sen.replace(" ik ", ' i know ')
    sen=sen.replace(" af ", ' as fuck ')
    sen=sen.replace(" hes ", ' he is ')
    sen=sen.replace(" des ", ' this ')
    sen=sen.replace(" bout ", ' about ')
    sen=sen.replace(" em ", ' them ')
    sen=sen.replace(" stg ", ' swear to god ')
    sen=sen.replace(" bj ", ' blow job ')
    sen=sen.replace(" ig ", ' i guess ')
    sen=sen.replace(" fagg ", ' faggot ')
    sen=sen.replace(" fag ", ' faggot ')
    sen=sen.replace(" fagot ", ' faggot ')
    sen=sen.replace(" tfw ", ' That Feel When ')
    sen=sen.replace(" wa ", ' was ')
    sen=sen.replace(" n ", ' and ')
    sen=sen.replace(" y0 ", ' you ')
    sen=sen.replace(" dat ", ' that ')
    sen=sen.replace(" yo ", ' you ')
    sen=sen.replace("youre ", ' you are ')
    sen=sen.replace(" v ", ' vagina ')
    sen=sen.replace(" pusy ", ' pussy ')
    sen=sen.replace(" pusy", ' pussy ')
    sen=sen.replace("pussie", 'pussy')
    sen=sen.replace(" v.", ' vagina ')
    sen=sen.replace(" doggystyle ", ' doggy style ')
    sen=sen.replace("di ck", 'dick')
    sen=sen.replace("r you", 'are you')
    sen=sen.replace("r u", 'are you')
    sen=sen.replace("f*ck", 'fuck')
    sen=sen.replace("f**", 'fuck')
    sen=sen.replace(" fxk ", 'fuck')
    sen=sen.replace(" wil ", 'will')
    sen=sen.replace(" il ", 'i will')
    sen=sen.replace(" asss ", ' ass ')
    sen=sen.replace(" lifeles ", ' lifeless ')
    sen=sen.replace(" ugler ", ' uglier ')
    sen=sen.replace(" you're ", ' you are ') 
    sen=sen.replace(",", '.') 
    sen=sen.replace("?", '.') 
    sen=sen.lower()
    for s in w:
        n=s+s+s
        sen=sen.replace(n, s)
    sen=" ".join(sen.split())
    QA.append(sen)
# print(QA)
q_a=QA
print(len(q_a))

11024


In [100]:
# Extention of emaning with pos tag
def disambiguation(sen):
    allWords=disambiguate(sen)
#     print(allWords)
    new_sen=[]
    for x in allWords:
        
        
        if x[0]=='fuck':
            for l in x[1].lemmas():
                if l.name() == 'fuck' or l.name() == 'fucking' or l.name() == 'screw' or l.name() == 'screwing' or l.name() == 'ass' or l.name() == 'nooky' or l.name() == 'nookie' or l.name() == 'piece_of_ass'  or l.name() == 'roll_in_the_hay' or l.name() == 'shag' or l.name() == 'shtup' or l.name() == 'sleep_together' or l.name() == 'roll_in_the_hay' or l.name() == 'make_out'  or l.name() == 'sleep_with' or l.name() == 'get_laid' or l.name() == 'have_sex'  or l.name() == 'be_intimate' or l.name() == 'have_intercourse'  or l.name() == 'screw' or l.name() == 'fuck' or l.name() == 'jazz' or l.name() == 'eff' or l.name() == 'hump' or l.name() == 'lie_with' or l.name() == 'bed'  or l.name() == 'bang'  or l.name() == 'bonk':
                    ns=sen.replace(x[0], l.name())
                    ns=ns.lower()
                    new_sen.append(ns.replace('_', ' '))
        elif x[0]=='love':
            for l in x[1].lemmas():
                if l.name()=='love' or l.name()=='passion' or l.name()=='beloved' or l.name()=='dearest' or l.name()=='honey' or l.name()=='know' or l.name()=='love_life' or l.name()=='have_a_go_at_it' or l.name()=='get_it_on':
                    ns=sen.replace(x[0], l.name())
                    ns=ns.lower()
                    new_sen.append(ns.replace('_', ' '))
        else:
            try: 
                pos=x[1].pos()
                for syn in wn.synsets(x[0]):
                    if syn.pos()==pos:
                        for l in syn.lemmas():
                            ns=sen.replace(x[0], l.name())
                            ns=ns.lower()
                            new_sen.append(ns.replace('_', ' '))

            except:
                ns=sen.lower()
                new_sen.append(ns.replace('_', ' '))
    new_sen = list(dict.fromkeys(new_sen))
    return new_sen



In [7]:
# Extention of emaning with all words
def disambiguation(sen):
    text = word_tokenize(sen)
    allWords=nltk.pos_tag(text)
    new_sen=[]
    for x in allWords:
        print(x)
#         if x[0]=='fuck':
#             synonyms = wn.synsets(x[0])
#             for syn in synonyms:
#                 for l in syn.lemmas():
#                     if l.name() == 'fuck' or l.name() == 'fucking' or l.name() == 'screw' or l.name() == 'screwing' or l.name() == 'ass' or l.name() == 'nooky' or l.name() == 'nookie' or l.name() == 'piece_of_ass'  or l.name() == 'roll_in_the_hay' or l.name() == 'shag' or l.name() == 'shtup' or l.name() == 'sleep_together' or l.name() == 'roll_in_the_hay' or l.name() == 'make_out'  or l.name() == 'sleep_with' or l.name() == 'get_laid' or l.name() == 'have_sex'  or l.name() == 'be_intimate' or l.name() == 'have_intercourse'  or l.name() == 'screw' or l.name() == 'fuck' or l.name() == 'jazz' or l.name() == 'eff' or l.name() == 'hump' or l.name() == 'lie_with' or l.name() == 'bed'  or l.name() == 'bang'  or l.name() == 'bonk':
#                         ns=sen.replace(x[0], l.name())
#                         ns=ns.lower()
#                         new_sen.append(ns.replace('_', ' '))
                        
#         elif x[0]=='love':
#             synonyms = wn.synsets(x[0])
#             for l in syn.lemmas():
#                 for syn in synonyms:
#                     for l in syn.lemmas():
#                         if l.name()=='love' or l.name()=='passion' or l.name()=='beloved' or l.name()=='dearest' or l.name()=='honey' or l.name()=='know' or l.name()=='love_life' or l.name()=='have_a_go_at_it' or l.name()=='get_it_on':
#                             ns=sen.replace(x[0], l.name())
#                             ns=ns.lower()
#                             new_sen.append(ns.replace('_', ' '))  

        try: 
            synonyms = wn.synsets(x[0])
            for syn in synonyms:
                for l in syn.lemmas():
                    ns=sen.replace(x[0], l.name())
                    ns=ns.lower()
                    new_sen.append(ns.replace('_', ' '))
        except:
            ns=sen.lower()
            new_sen.append(ns.replace('_', ' '))
    new_sen = list(dict.fromkeys(new_sen))
    return new_sen
disambiguation('I love her')

('I', 'PRP')
('love', 'VBP')
('her', 'PRP$')


['iodine love her',
 'iodin love her',
 'i love her',
 'atomic number 53 love her',
 'one love her',
 '1 love her',
 'ace love her',
 'single love her',
 'unity love her',
 'ane love her',
 'i passion her',
 'i beloved her',
 'i dear her',
 'i dearest her',
 'i honey her',
 'i sexual love her',
 'i erotic love her',
 'i lovemaking her',
 'i making love her',
 'i love life her',
 'i enjoy her',
 'i sleep together her',
 'i roll in the hay her',
 'i make out her',
 'i make love her',
 'i sleep with her',
 'i get laid her',
 'i have sex her',
 'i know her',
 'i do it her',
 'i be intimate her',
 'i have intercourse her',
 'i have it away her',
 'i have it off her',
 'i screw her',
 'i fuck her',
 'i jazz her',
 'i eff her',
 'i hump her',
 'i lie with her',
 'i bed her',
 'i have a go at it her',
 'i bang her',
 'i get it on her',
 'i bonk her']

In [11]:
# Extention of meaning with disambiguation
def disambiguation(sen):
    allWords=disambiguate(sen)
    print(allWords)
    new_sen=[]
    for x in allWords:
        if x[0]=='fuck':
            for l in x[1].lemmas():
                if l.name() == 'fuck' or l.name() == 'fucking' or l.name() == 'screw' or l.name() == 'screwing' or l.name() == 'ass' or l.name() == 'nooky' or l.name() == 'nookie' or l.name() == 'piece_of_ass'  or l.name() == 'roll_in_the_hay' or l.name() == 'shag' or l.name() == 'shtup' or l.name() == 'sleep_together' or l.name() == 'roll_in_the_hay' or l.name() == 'make_out'  or l.name() == 'sleep_with' or l.name() == 'get_laid' or l.name() == 'have_sex'  or l.name() == 'be_intimate' or l.name() == 'have_intercourse'  or l.name() == 'screw' or l.name() == 'fuck' or l.name() == 'jazz' or l.name() == 'eff' or l.name() == 'hump' or l.name() == 'lie_with' or l.name() == 'bed'  or l.name() == 'bang'  or l.name() == 'bonk':
                    ns=sen.replace(x[0], l.name())
                    ns=ns.lower()
                    new_sen.append(ns.replace('_', ' '))
        elif x[0]=='love':
            for l in x[1].lemmas():
                if l.name()=='love' or l.name()=='passion' or l.name()=='beloved' or l.name()=='dearest' or l.name()=='honey' or l.name()=='know' or l.name()=='love_life' or l.name()=='have_a_go_at_it' or l.name()=='get_it_on':
                    ns=sen.replace(x[0], l.name())
                    ns=ns.lower()
                    new_sen.append(ns.replace('_', ' '))
        else:
            try: 
                for l in x[1].lemmas():
                        ns=sen.replace(x[0], l.name())
                        ns=ns.lower()
                        new_sen.append(ns.replace('_', ' '))
            except:
                ns=sen.lower()
                new_sen.append(ns.replace('_', ' '))
        new_sen = list(dict.fromkeys(new_sen))
    return new_sen

disambiguation('I hate her')

[('I', None), ('hate', Synset('hate.v.01')), ('her', None)]


['i hate her', 'i detest her']

In [101]:
new_q_a=[]
new_label=[]
for i in  range(len(q_a)):   
    collction_sen= disambiguation(q_a[i])
    for x in collction_sen:
        new_q_a.append(x)
        new_label.append(label[i])

In [76]:
# def extract_meanings(word):
#     synonyms = []
#     # Get all possible meanings of the word
#     for syn in wn.synsets(word):
#         print(syn)
#         for l in syn.lemmas():
#             print(l)
#             synonyms.append(l.name())
#     return synonyms
# word = input("type a word to retrieve all other words with similar meaning: ")
# synoym = extract_meanings(word)
    

In [102]:
print(len(new_q_a))
print(len(new_label))

618155
618155


In [79]:
my_dict = { 'q_a':new_q_a,'label':new_label}
df = pd.DataFrame(my_dict)
# df.to_csv('expansation_semantinc_meaning_using_postag',index=False)

In [103]:
# Read data files
comments_attack=pd.DataFrame()
comments_attack['comment']=new_q_a
comments_attack['label']=new_label

In [104]:
dataframe = comments_attack

# Text preprocessing
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace("NEWLINE_TOKEN", " "))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace("TAB_TOKEN", " "))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.lower())
# dataframe['comment'] = dataframe['comment'].apply((lambda x: re.sub('[/(){}\[\]\|@,;]','',x)))
# dataframe['comment'] = dataframe['comment'].apply((lambda x: re.sub('[^0-9a-z #+_]','',x)))
# dataframe['comment'] = dataframe['comment'].apply((lambda x: re.sub(' +',' ',x)))



In [87]:
#this code will remove stop word
for c in stop: #stop word removing
    dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(' '+c+' ', ' '))


NameError: name 'stop' is not defined

In [105]:
print(dataframe['label'])

0         1
1         1
2         1
3         1
4         1
         ..
618150    0
618151    0
618152    0
618153    0
618154    0
Name: label, Length: 618155, dtype: int64


In [106]:
print(dataframe['comment'])

0                 fuck justin bieber. he's a fucking fag
1           fucking justin bieber. he's a fuckinging fag
2               screw justin bieber. he's a screwing fag
3         screwing justin bieber. he's a screwinging fag
4                   ass justin bieber. he's a assing fag
                               ...                      
618150                           hunting watch hayes omg
618151                            hunter helen hayes omg
618152                    hunter rutherford b. hayes omg
618153              hunter rutherford birchard hayes omg
618154                        hunter president hayes omg
Name: comment, Length: 618155, dtype: object


In [107]:
#split the data into training and validation sets
train_x, valid_x, train_y, valid_y = train_test_split(dataframe['comment'], dataframe['label'], test_size=0.2, random_state=42)

In [108]:
print('train comments length: ',len(train_x))
print('test comments length: ',len(valid_x))

train comments length:  494524
test comments length:  123631


In [109]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'.')
count_vect.fit(dataframe['comment'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [110]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(dataframe['comment'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# word level ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(3,4), max_features=5000)
tfidf_vect_ngram.fit(dataframe['comment'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(dataframe['comment'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x)

#feartures nlp
# dataframe['char_count'] = dataframe['comment'].apply(len)
# dataframe['word_count'] = dataframe['comment'].apply(lambda x: len(x.split()))
# dataframe['word_density'] = dataframe['char_count'] / (dataframe['word_count']+1)
# dataframe['punctuation_count'] = dataframe['comment'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
# dataframe['title_word_count'] = dataframe['comment'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
# dataframe['upper_case_word_count'] = dataframe['comment'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))


In [111]:
# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('data/wiki-news-300d-1M.vec', encoding="utf8")):
    values = line.split()
    embeddings_index[values[0]] = np.asarray(values[1:], dtype='float32')

# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(dataframe['comment'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure e
# qual length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

# create token-embedding mapping
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [112]:
def train_model(classifier, xtrain, ytrain, xvalid, yvalid): 
    # fit the training dataset on the classifier
    classifier.fit(xtrain, ytrain)
    # predict the labels on validation dataset
    predictions = classifier.predict(xvalid)     
    accuracy = metrics.accuracy_score(predictions, yvalid)
    f1score = metrics.f1_score(yvalid, predictions, average='weighted')
    return accuracy, f1score

In [113]:
# Naive Bayes on Count Vectors
# accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count, valid_y)
# print("NB, Count Vectors:   accuracy: %s      f1 score: %s"% (accuracy,f1score))

# Naive Bayes on Word Level TF IDF Vectors
accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf, valid_y)
print("NB, WordLevel TF-IDF:   accuracy: %s     f1 score: %s"% (accuracy,f1score))

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, valid_y)
print("NB, N-Gram Vectors:   accuracy: %s     f1 score: %s"% (accuracy,f1score))

# Naive Bayes on Character Level TF IDF Vectors
accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars, valid_y)
print("NB, CharLevel Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

NB, WordLevel TF-IDF:   accuracy: 0.9058165023335571     f1 score: 0.8955371431323509
NB, N-Gram Vectors:   accuracy: 0.830002183918273     f1 score: 0.8023529375402864
NB, CharLevel Vectors:   accuracy: 0.8797469890237886   f1 score: 0.8616585232974142


In [114]:
# Linear Classifier on Count Vectors
# accuracy, f1score = train_model(linear_model.LogisticRegression(), xtrain_count, train_y, xvalid_count, valid_y)
# print("LR, Count Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

# Linear Classifier on Word Level TF IDF Vectors
accuracy, f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf, valid_y)
print("LR, WordLevel TF-IDF:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy, f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, valid_y)
print("LR, N-Gram Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

# Linear Classifier on Character Level TF IDF Vectors
accuracy, f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars, valid_y)
print("LR, CharLevel Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LR, WordLevel TF-IDF:   accuracy: 0.9586026158487758   f1 score: 0.9580267970325165
LR, N-Gram Vectors:   accuracy: 0.8471257208952447   f1 score: 0.8104175998950054
LR, CharLevel Vectors:   accuracy: 0.9576562512638416   f1 score: 0.9569269222435989


In [115]:
# convert to one_hot
train_y_onehot = keras.utils.to_categorical(train_y, 3)
valid_y_onehot = keras.utils.to_categorical(valid_y, 3)

In [116]:
def cnn(xtrain, ytrain, xvalid, yvalid, epochs = 3):
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(3, activation="softmax")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer='adam', loss='categorical_crossentropy',  metrics=['accuracy'])
    model.fit(xtrain, ytrain,
              batch_size=256,
              epochs=epochs)
    predictions = model.predict(xvalid)
    predictions = predictions.argmax(axis=-1)
    accuracy = model.evaluate(xvalid, yvalid, verbose=0)
    f1score = metrics.f1_score(valid_y, predictions, average='weighted')
    return accuracy, f1score

In [ ]:
accuracy, f1score = cnn(train_seq_x, train_y_onehot, valid_seq_x, valid_y_onehot)
print("CNN, Word Embeddings acuuracy accuracy:%s     f1 score: %s"% (accuracy[1], f1score))

Epoch 1/3
494524/494524 [==============================] - 930s 2ms/step - loss: 0.1305 - accuracy: 0.9508
Epoch 2/3
494524/494524 [==============================] - 931s 2ms/step - loss: 0.0454 - accuracy: 0.9838
Epoch 3/3
494524/494524 [==============================] - 1156s 2ms/step - loss: 0.0345 - accuracy: 0.9877


In [ ]:
def lstm(xtrain, ytrain, xvalid, yvalid, epochs = 1):
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer1 = layers.LSTM(128)(embedding_layer)
    dropout1 = layers.Dropout(0.5)(lstm_layer1)
    #lstm_layer2 = layers.LSTM(128)(dropout1)
    #dropout2 = layers.Dropout(0.5)(lstm_layer2)
    # Add the output Layers
    output_layer = layers.Dense(3, activation="softmax")(dropout1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(xtrain, ytrain,
              batch_size=256,
              epochs=3)
    
    predictions = model.predict(xvalid)
    predictions = predictions.argmax(axis=-1)
    accuracy = model.evaluate(xvalid, yvalid, verbose=0)
    f1score = metrics.f1_score(valid_y, predictions, average='weighted')
    return accuracy, f1score

In [ ]:
accuracy, f1score = lstm(train_seq_x, train_y_onehot, valid_seq_x, valid_y_onehot)
print("LSTM, Word Embeddings accuracy:%s     f1 score: %s"% (accuracy[1], f1score))

In [ ]:
(i)  Load each Sentance
(ii) Perform word tokenize, make a list of words
(ii) FOR each word, disambiguate and find sense specific Synset
        IF sense sense spefic Synset has Synonyms
            FOR each synonym
                Replace sentance word with  synonym and generate new sentance
            END FOR
        END IF
    END FOR
    END